#### Import modules

In [1]:
print("Import modules")

Import modules


In [2]:
%config Completer.use_jedi = False

In [3]:
try:
    import talos
except:
    # Install pip package in the current Jupyter kernel
    import sys
    #!{sys.executable} -m pip install talos

In [4]:
try:
    import matplotlib_venn
except:
    # Install pip package in the current Jupyter kernel
    import sys
    #!{sys.executable} -m pip install matplotlib-venn

In [5]:
try:
    import swifter
except:
    # Install pip package in the current Jupyter kernel
    import sys
    #!{sys.executable} -m pip install swifter
    

In [6]:
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, Conv1D
# from tensorflow.keras.models import Model
# from tensorflow.keras import backend as K
# from tensorflow.keras import Sequential


from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, ShuffleSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

from scipy.stats import kendalltau, kendalltau
from scipy.sparse import csr_matrix

from multiprocessing import Pool, cpu_count

import sys
import math
import time
import pickle
import random

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from shutil import rmtree
from datetime import datetime, timedelta
from os import listdir, mkdir
from os.path import isfile, join
from itertools import compress

import ipywidgets as widgets
from IPython.display import HTML, display

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize, LogNorm
from scipy.interpolate import interpn
from scipy.stats import spearmanr

from decimal import Decimal

# import tensorflow as tf

from IPython.display import clear_output, display

from matplotlib_venn import venn2

from collections import Counter


In [7]:
#print(tf.version)

In [8]:
random.seed(a=42)
sns.set_context("notebook", font_scale=1.4)
%matplotlib inline

refTablesPath   = "_01_RefTables/"
rawDataPath     = "_02_RawData/"
encodedDataPath = "_03_EncodedDatasets/"
cachedDataPath  = "_03_CachedData"
modelsPath      = "_04_Models/"
figuresPath     = "_08_Figures/"
geneticAlgo_output_folder="_07_GeneticAlgo_Results/"

for f in [refTablesPath,
          rawDataPath,
          encodedDataPath,
          modelsPath,
          figuresPath,
          geneticAlgo_output_folder]:
    

    try:mkdir(f)
    except:pass

#### Define functions

In [9]:
print("Define functions")

Define functions


In [10]:
# function to view encoded proteins
def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))
    
def flatten(list_of_lists):
    return([item for sublist in list_of_lists for item in sublist])

def print_list(l):
    print(", ".join([str(elt) for elt in l]))

In [11]:

def density_scatter( x , y, ax = None, sort = True, bins = 20,norm_min=None,norm_max=None, **kwargs )   :
    """
    Scatter plot colored by 2d histogram
    """
    if ax is None :
        fig , ax = plt.subplots(figsize=(11, 10))
        fig.patch.set_facecolor('w')
    data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
    z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

    #To be sure to plot all data
    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    if sort :
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    ax.scatter( x, y, c=z, **kwargs )
    
    if norm_min!=None and norm_max!=None:
        print("norm!")
        norm = Normalize(vmin = norm_min, vmax = norm_max)
    else:
        norm = Normalize(vmin = np.min(z), vmax = np.max(z))
        
    cbar = fig.colorbar(cm.ScalarMappable(norm = norm), ax=ax)
    cbar.ax.set_ylabel('Density')

    return ax

In [12]:
# format genotypes

def format_genotypes(df):
    # remove S(ubstitution) at the beginning of each mutation
    result = pd.DataFrame([str(row[genotype_col]).replace(":S",":")[1:] if row[genotype_col]!="wt" else "wt" for index,row in df.iterrows()],
                          columns=[genotype_col],
                          index = df.index)
    # re-number positions so that initial M=0 (Louisa's numbering)
    def re_number(genotype):
        if genotype=="wt":
            pass
        else:
            shift=1
            genotype = ":".join(["".join((elt[0],str(int(elt[1:-1])+shift),elt[-1])) for elt in genotype.split(":")])
        return(genotype)
    
    
    result[genotype_col] = result[genotype_col].apply(lambda x:re_number(x))
    
    return(result)

get_formatted_genotypes = FunctionTransformer(format_genotypes)


In [13]:
# Recreate mutated seq from wt_seq + aaMutations

def genotype_to_seq(df):
    
    verbose=False
    
    new_seqs = []
    # genotype to seq
    for index, row in df.iterrows():
        
        new_seq = str(wt_seq)
        
        if row[genotype_col]=="wt":
            aaMutations=[]
        else:
            aaMutations = [(elt[0], int(elt[1:-1]), elt[-1]) for elt in str(row[genotype_col]).split(":")]
                
        if verbose:
            print(new_seq)
            print(len(new_seq))
            print(aaMutations)

        for elt in aaMutations:
            new_seq = new_seq[0:elt[1]]+elt[2]+new_seq[elt[1]+1:] 
            if verbose: print("*"*elt[1])
        
        if verbose:
            print(new_seq)
            print(len(new_seq))
            print("\n\n\n")

        new_seqs.append(new_seq)        

    new_seqs = pd.DataFrame(new_seqs,
                            columns=["full_seq"],
                            index=df.index)

    return(new_seqs)

# wrap for pipeline
get_seq_from_genotype = FunctionTransformer(genotype_to_seq)


In [14]:
# one hot encoding
# or use sklearn OH instead
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', "*"]

codes_as_dict = {codes[k]: [0]*len(codes) for k in range(len(codes))}

for k in range(len(codes)):
    codes_as_dict[codes[k]][k] = 1

def one_hot_encoder(df):
    # can encode the whole df
    # but a parallel version is defined below
    encoded_list = [flatten([codes_as_dict[k] for k in str(
        row["full_seq"])]) for index, row in df.iterrows()]

    cols = []
    for i in range(len(df.iloc[0]["full_seq"])):
        for c in codes:
            cols.append("Pos"+str(i)+c)

    encoded_one_hot = pd.DataFrame(encoded_list,
                                   index=df.index,
                                   columns=cols)

    return(encoded_one_hot)


In [15]:
def one_hot_encoder_pl(df):
    
    # count available cpus
    partitions = cpu_count()
    # split df in as many parts
    data_split = np.array_split(df, partitions)
    # define computing pool and get results
    pool = Pool(partitions)
    encoded_one_hot = pd.concat(pool.map(one_hot_encoder, data_split))
    pool.close()
    pool.join()
    
    return(encoded_one_hot)

# wrap for pipeline
get_one_hot_encoding_pl = FunctionTransformer(one_hot_encoder_pl)


In [16]:
import seaborn as sns

def makenicegraph(data=None, x=None, y=None, cmap='mako', bins=100, figsize=(5,5), snsbins=60,plot_clim=None):
    
    sns.set_style("white")
    
    ax1 = sns.jointplot(x=data[x],
                        y=data[y],
                        marginal_kws={"bins":snsbins,
                                      "color":"#1B80A2"},
                        ratio=5,
                        height=5,
                        xlim=(0,10),
                       ylim=(0,10))
    
    
    ax1.fig.set_size_inches(figsize[0], figsize[1])
    ax1.ax_joint.cla()
    plt.sca(ax1.ax_joint)
    plt.hist2d(data[x],data[y],bins,norm=LogNorm(),cmap=cmap)
    plt.xlim(plot_range)
    plt.ylim(plot_range)
    plt.plot(plot_range,plot_range, 'k-')
    
    plt.xlabel(x,fontsize=12)
    plt.ylabel(y,fontsize=12)

    cbar_ax = ax1.fig.add_axes([1, 0.25, .01, .5])
    cb = plt.colorbar(cax=cbar_ax)
    cb.set_label(r'$\log_{10}$ density of points',fontsize=13)
    
    if plot_clim!=None:
        plt.clim(plot_clim[0],plot_clim[1])


